In [82]:
# read it in to inspect it
with open('train_medical.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [83]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  17811683


In [42]:
# let's look at the first 1000 characters
print(text[:1000])

﻿   Title: Dorothy and the Wizard in Oz

Author: L. Frank Baum

Illustrator: John R. Neill

Language: English


*** START OF THE PROJECT GUTENBERG EBOOK DOROTHY AND THE WIZARD IN OZ ***



  [Illustration: DOROTHY AND THE WIZARD]

  [Illustration: PICKING THE PRINCESS.]




  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW YORK




  DEDICATED TO HARRIET A. B. NEAL.


         *       *       *       *       *


To My Readers


It's no use; no use at all. The children won't let me stop telling tales
of the Land of Oz. I know lots of other stories, and I hope to tell
them, some time or another; but just now my loving tyrants won't allow
me. They cry: "Oz--Oz! more about Oz, Mr. Baum!" and what can I do but
obey their commands?

This is Our Book--mine and the children's. For they have flooded me with
thousands of suggestions in regard to it

In [43]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"&'()*,-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz﻿
81


In [44]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[61, 62, 62, 1, 73, 61, 58, 71, 58]
hii there


In [45]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([232456]) torch.int64
tensor([80,  1,  1,  1, 44, 62, 73, 65, 58, 22,  1, 28, 68, 71, 68, 73, 61, 78,
         1, 54, 67, 57,  1, 73, 61, 58,  1, 47, 62, 79, 54, 71, 57,  1, 62, 67,
         1, 39, 79,  0,  0, 25, 74, 73, 61, 68, 71, 22,  1, 36, 11,  1, 30, 71,
        54, 67, 64,  1, 26, 54, 74, 66,  0,  0, 33, 65, 65, 74, 72, 73, 71, 54,
        73, 68, 71, 22,  1, 34, 68, 61, 67,  1, 42, 11,  1, 38, 58, 62, 65, 65,
         0,  0, 36, 54, 67, 60, 74, 54, 60, 58, 22,  1, 29, 67, 60, 65, 62, 72,
        61,  0,  0,  0,  8,  8,  8,  1, 43, 44, 25, 42, 44,  1, 39, 30,  1, 44,
        32, 29,  1, 40, 42, 39, 34, 29, 27, 44,  1, 31, 45, 44, 29, 38, 26, 29,
        42, 31,  1, 29, 26, 39, 39, 35,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25,
        38, 28,  1, 44, 32, 29,  1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39,
        50,  1,  8,  8,  8,  0,  0,  0,  0,  1,  1, 51, 33, 65, 65, 74, 72, 73,
        71, 54, 73, 62, 68, 67, 22,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38,
       

In [46]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [47]:
block_size = 8
train_data[:block_size+1]

tensor([80,  1,  1,  1, 44, 62, 73, 65, 58])

In [48]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([80]) the target: 1
when input is tensor([80,  1]) the target: 1
when input is tensor([80,  1,  1]) the target: 1
when input is tensor([80,  1,  1,  1]) the target: 44
when input is tensor([80,  1,  1,  1, 44]) the target: 62
when input is tensor([80,  1,  1,  1, 44, 62]) the target: 73
when input is tensor([80,  1,  1,  1, 44, 62, 73]) the target: 65
when input is tensor([80,  1,  1,  1, 44, 62, 73, 65]) the target: 58


In [49]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[61, 58,  1, 59, 68, 68, 65, 62],
        [71, 71, 62, 75, 54, 65, 72,  9],
        [ 1, 76, 54, 72,  1, 66, 74, 56],
        [ 1, 59, 68, 71,  1, 59, 62, 60]])
targets:
torch.Size([4, 8])
tensor([[58,  1, 59, 68, 68, 65, 62, 72],
        [71, 62, 75, 54, 65, 72,  9,  1],
        [76, 54, 72,  1, 66, 74, 56, 61],
        [59, 68, 71,  1, 59, 62, 60, 61]])
----
when input is [61] the target: 58
when input is [61, 58] the target: 1
when input is [61, 58, 1] the target: 59
when input is [61, 58, 1, 59] the target: 68
when input is [61, 58, 1, 59, 68] the target: 68
when input is [61, 58, 1, 59, 68, 68] the target: 65
when input is [61, 58, 1, 59, 68, 68, 65] the target: 62
when input is [61, 58, 1, 59, 68, 68, 65, 62] the target: 72
when input is [71] the target: 71
when input is [71, 71] the target: 62
when input is [71, 71, 62] the target: 75
when input is [71, 71, 62, 75] the target: 54
when input is [71, 71, 62, 75, 54] the target: 65
when input is [

In [50]:
print(xb) # our input to the transformer

tensor([[61, 58,  1, 59, 68, 68, 65, 62],
        [71, 71, 62, 75, 54, 65, 72,  9],
        [ 1, 76, 54, 72,  1, 66, 74, 56],
        [ 1, 59, 68, 71,  1, 59, 62, 60]])


In [51]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 81])
tensor(4.7797, grad_fn=<NllLossBackward0>)

﻿P,?vAP,(MZoe1uZ7z("w
3)!y﻿p7843OV349B GpThAm4BlLV_Z 8?M3[mX
nh4Yv'()DR)D5w3oEg[P*FdMZROFIjBQ[-w:g52


In [52]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [53]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.674656867980957


In [54]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


vh;oITRP7_DoniPtUXjhO83O8 j6Fk&?(Zj"xT(RAU"﻿9;s(!f[f,_K8
iKNNtf5C"A,Mtfsf5mStfgtT[.da
﻿
XxLR6DY7_VK:)OAQ,)cb4yMZ"R*﻿0Bk5Se uW73oKM;m4m"485;)qN0Od]4Ju9fz52O;&2B34m
ZYtVu?a?9yy5j
RODD_ZL?iS,c;!;ayxO&?*5D_(VqK﻿oR[dS8sT-
W*y27;5VuR3RY6xmgxIp'45PgdB6';dlkEa﻿!7D(JSnMVu9F_8a(C]&w
aKS)i,BlDRtlz'wNkbxNQ(eGGGra,Uh5*5s7_rCtT*5r6D*zL Y7cbJ&
T*5P,*5JnZ?IZj2 DQB8qLW)f;[_;8Zus&u&[0Y-XVhUPsho2tFr-o'HL)p"3D-p&N)hKK']SxFIjhYsihY9[L!d1uUU)!CIT&O)
swbq0Bei!a3Ox[j-;*3gCS,Eh"u.sU'O8gVWG6d1O;WF]N0'SLed5:1&Kv]1
amLR1:2


In [55]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [56]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [57]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [58]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [59]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

False

In [60]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [61]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [62]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [63]:
k.var()

tensor(1.0449)

In [64]:
q.var()

tensor(1.0700)

In [65]:
wei.var()

tensor(1.0918)

In [66]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [67]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [68]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [69]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [70]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [71]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



In [86]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('train_medical.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.212438 M parameters
step 0: train loss 4.5851, val loss 4.5840
step 100: train loss 2.7187, val loss 2.7380
step 200: train loss 2.5661, val loss 2.5869
step 300: train loss 2.5088, val loss 2.5091
step 400: train loss 2.4430, val loss 2.4420
step 500: train loss 2.3492, val loss 2.3490
step 600: train loss 2.2786, val loss 2.2913
step 700: train loss 2.2395, val loss 2.2398
step 800: train loss 2.1702, val loss 2.1862
step 900: train loss 2.1365, val loss 2.1463
step 1000: train loss 2.1069, val loss 2.1046
step 1100: train loss 2.0803, val loss 2.0709
step 1200: train loss 2.0490, val loss 2.0552
step 1300: train loss 2.0154, val loss 2.0093
step 1400: train loss 1.9980, val loss 2.0087
step 1500: train loss 1.9860, val loss 1.9903
step 1600: train loss 1.9576, val loss 1.9610
step 1700: train loss 1.9446, val loss 1.9399
step 1800: train loss 1.9092, val loss 1.9162
step 1900: train loss 1.9104, val loss 1.9005
step 2000: train loss 1.8801, val loss 1.8909
step 2100: train loss 1.